In [1]:
import numpy as np
import pandas as pd
import math
import datetime as dt
import datapackage
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import multinomial
from typing import List
import seaborn as sns

## system characterisation ##

class State: 
    def __init__(self,mean,eta,lambda_switch,sigma):
        self.m = mean
        self.e = eta 
        self.l = lambda_switch
        self.s = sigma

M = 2
N = 2

state_1 = State(4,0.2,0.1,0.15)

def Profit(E,P,vc,fc,pi_tax,c_tax):
    pi = ((E*(P-vc-c_tax))-fc)*(1-pi_tax)
    return pi

T = 2
E = 2000
r = 0.0125 
vc = 1
fc = 200
pi_tax = 0.2
c_tax = 0.4

switch = (0.2,0.6)

## node construction ##

class node:
    def __init__(self,value,temp):
        self.value = v
        self.temp = t

## finite difference parameter definition ##

## parameter defn for P_i in i = 2,...,i_max-1 ##

def difference(P,p_step,sigma,eta,mean):
    difference = np.zeros(2)
    a_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_central = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    b_i_forward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    a_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step) - eta*(mean-P)/(p_step*2)
    b_i_backward = (sigma**2)*(P**2)/((p_step*2)*p_step)
    if a_i_central > 0:
        difference[0] = a_i_central
    else:
        difference[0] = a_i_forward
    if b_i_central > 0:
        difference[1] = b_i_central
    else:
        difference[1] = b_i_backward
    return(difference)

## parameter defn for P_i in i = 1 ##
    
def difference_min(P,p_step,eta,mean):
    a_i = 0
    b_i = eta*(mean-P)/(p_step)
    return [a_i,b_i]

## parameter defn for P_i in i = i_max ##

def difference_max(P,p_step,eta,mean):
    a_i = eta*(mean - P)/(p_step)
    b_i = 0
    return [a_i,b_i]

## lattice parameters ##

p_step = 1
s0_step = 100000

p_1 = 0.5
p_max = 8
p_steps = int((p_max-p_1)/p_step)

s0_1 = 100000
s0_max = 500000

## lattice construction ##

P_grid_lower = list(range(5,40,p_steps))
P_grid_middle = list(range(41,66,1))
P_grid_upper = list(range(68,80,2))
P_grid_high = list(range(85,110,5))
P_grid_vhigh = list(range(120,160,10))

P_grid = [0.5,1,1.5,2,2.5,3,3.5,4,5,7]

s0_grid = list(range(s0_1,s0_max,s0_step))

temp_node = np.full((len(P_grid),len(s0_grid),M,N),0)
node_value = np.full((len(P_grid),len(s0_grid),M,N),0)
Lv = np.full((len(P_grid),len(s0_grid),M,N),0)

## here we store the optimal state n in N conditional on prior state m in M ## 

temp_path = np.full((len(P_grid),len(s0_grid),M),0)
opt_path = []
 
cost = np.zeros((2,2))
temp_compare = np.zeros((2))
        
cost[0,0] = 0
cost[1,0] = 2000
cost[1,1] = 0
cost[0,1] = 2000

In [2]:
for t in range(30):
    for q in range(4):
        
## M refers to adjacent state, N refers to current state ##
        
## Quarterly profit at each node ## 
        
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        if n == 1:
                            if (120 - (t*4 + q))*E < s0_grid[k]:
                                temp_node[i,k,m,n] = temp_node[i,k,m,n] + Profit(E,P_grid[i],vc,fc,pi_tax,c_tax)
                            else:
                                temp_node[i,k,m,n] = temp_node[i,k,m,n] - fc
                        else:
                            temp_node[i,k,m,n] = temp_node[i,k,m,n] - fc
        
## Lagrange Differential at each node ##
        
## then we loop over lagrange differential for nodes inside the boundary ##
    
        for i in range(1,len(P_grid) - 1):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        diff = difference(P_grid[i],p_step,state_1.s,state_1.e,state_1.m)
                        Lv[i,k,m,n] = diff[0]*temp_node[i - 1,k,m,n] + diff[1]*temp_node[i + 1,k,m,n] - (diff[0]+diff[1]+r)*temp_node[i,k,m,n] 

## we loop over Lagrange differential for boundary nodes ##                
    
        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    Lv[0,k,m,n] = (-1)*r*temp_node[0,k,m,n]

        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    Lv[len(P_grid) - 1,k,m,n] = (-1)*r*temp_node[len(P_grid) - 1,k,m,n]
                    
## then we apply Lv to temporary node values ##
        for i in range(len(P_grid)):
            for k in range(len(s0_grid)):
                for m in range(M):
                    for n in range(N):
                        temp_node[i,k,m,n] = temp_node[i,k,m,n] + Lv[i,k,m,n]
                        

## we apply conditional costs on each state ##                           
                            
    for i in range(len(P_grid)):
        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    temp_node[i,k,m,n] = temp_node[i,k,m,n] - cost[m,n]

## we identify the optimal conditional state n in N for every adjacent state m in M ##

    for i in range(len(P_grid)):
        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    temp_compare[n] = temp_node[i,k,m,n]
                temp_path[i,k,m] = np.argmax(temp_compare)

## we for the next cycle, we adjust option value ##

    for i in range(len(P_grid)):
        for k in range(len(s0_grid)):
            for m in range(M):
                for n in range(N):
                    node_value[i,k,m,n] = temp_node[i,k,m,(temp_path[i,k,m])]
                    temp_node[i,k,m,n] = node_value[i,k,m,n]
                
                

In [3]:
print(Lv)

[[[[  156   224]
   [  374   442]]

  [[  156   224]
   [  500   568]]

  [[  156   224]
   [  532   601]]

  [[  156   224]
   [  532   601]]]


 [[[  198   227]
   [  497   527]]

  [[  212   241]
   [  604   633]]

  [[  208   237]
   [  619   649]]

  [[  208   237]
   [  619   649]]]


 [[[  339   329]
   [  147   137]]

  [[  301   292]
   [  103    93]]

  [[  276   266]
   [   81    71]]

  [[  276   266]
   [   81    71]]]


 [[[  145    96]
   [ -220  -269]]

  [[  -21   -70]
   [ -455  -504]]

  [[  -80  -129]
   [ -525  -574]]

  [[  -80  -129]
   [ -525  -574]]]


 [[[ -265  -353]
   [ -563  -651]]

  [[ -602  -690]
   [-1003 -1091]]

  [[ -701  -790]
   [-1124 -1212]]

  [[ -701  -790]
   [-1124 -1212]]]


 [[[ -669  -797]
   [ -894 -1022]]

  [[-1196 -1323]
   [-1544 -1671]]

  [[-1338 -1465]
   [-1719 -1846]]

  [[-1338 -1465]
   [-1719 -1846]]]


 [[[ -660  -750]
   [ -884  -974]]

  [[-1583 -1673]
   [-1929 -2019]]

  [[-1830 -1921]
   [-2210 -2300]]

  [[-1830 -1921]

In [4]:
print(node_value)

[[[[-12337 -12337]
   [-31570 -31570]]

  [[-12337 -12337]
   [-41514 -41514]]

  [[-12337 -12337]
   [-44087 -44087]]

  [[-12337 -12337]
   [-44087 -44087]]]


 [[[-11439 -11439]
   [-26625 -26625]]

  [[ -8760  -8760]
   [-30140 -30140]]

  [[ -7608  -7608]
   [-30510 -30510]]

  [[ -7608  -7608]
   [-30510 -30510]]]


 [[[ -5730  -5730]
   [ -3032  -3032]]

  [[  3744   3744]
   [  5196   5196]]

  [[  6943   6943]
   [  7963   7963]]

  [[  6943   6943]
   [  7963   7963]]]


 [[[ 11927  11927]
   [ 24739  24739]]

  [[ 31345  31345]
   [ 47032  47032]]

  [[ 37146  37146]
   [ 53419  53419]]

  [[ 37146  37146]
   [ 53419  53419]]]


 [[[ 36568  36568]
   [ 53491  53491]]

  [[ 67632  67632]
   [ 90868  90868]]

  [[ 76330  76330]
   [101121 101121]]

  [[ 76330  76330]
   [101121 101121]]]


 [[[ 63104  63104]
   [ 82613  82613]]

  [[106586 106586]
   [135536 135536]]

  [[118329 118329]
   [149775 149775]]

  [[118329 118329]
   [149775 149775]]]


 [[[111225 111225]
   [13076